In [3]:
import nltk
import pandas as pd
from nltk.corpus import stopwords, wordnet

nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\timot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\timot\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\timot\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [4]:
test_df = pd.read_json('test.jsonl', lines=True)
X_test = test_df['string']
y_test = test_df['label']

test_df

,source,citeEnd,sectionName,citeStart,string,label,citingPaperId,citedPaperId,isKeyCitation,id,unique_id,excerpt_index,label2,label_confidence
0,acronym,31.0,,28.0,"Chapel, as well as X10 [2], UPC [3] , CoArray ...",background,2c6797dab4c118cb73197f65ba39dacc99ac743d,95c37bc99982d33873fd141ee00857160fd717a0,True,2c6797dab4c118cb73197f65ba39dacc99ac743d>95c37...,2c6797dab4c118cb73197f65ba39dacc99ac743d>95c37...,6,NaN,NaN
1,explicit,166.0,Discussion,156.0,"In addition, the result of the present study s...",result,fa7145adc9f8cfb8af7a189d9040c13c84ced094,20e23b4f76761d246a7c3b00b80e139e2008f77d,False,fa7145adc9f8cfb8af7a189d9040c13c84ced094>20e23...,fa7145adc9f8cfb8af7a189d9040c13c84ced094>20e23...,0,supportive,NaN
2,explicit,145.0,Discussion,137.0,Several instruments that more specifically add...,background,98a8d8c0c5dae246720d4f339b88e8a9f44e3002,bd222c7ec83dadefba513738290b3624f6dd6b21,True,98a8d8c0c5dae246720d4f339b88e8a9f44e3002>bd222...,98a8d8c0c5dae246720d4f339b88e8a9f44e3002>bd222...,1,NaN,NaN
3,explicit,158.0,Methods,137.0,Organotypic hippocampal slice cultures\nInterf...,method,aeb178ef1910a61152cd74209c28641199c82855,754c04953c261072fa367f4104e3deff082d9484,False,aeb178ef1910a61152cd74209c28641199c82855>754c0...,aeb178ef1910a61152cd74209c28641199c82855>754c0...,1,NaN,1.0
4,explicit,186.0,RESULTS,178.0,Activated PBMC are the basis of the standard P...,background,e4d2591ac3bb65e2ec59f092884a7b15b8018592,f0fb468a54fe8021bc7986a1618222c4fcd16df4,False,e4d2591ac3bb65e2ec59f092884a7b15b8018592>f0fb4...,e4d2591ac3bb65e2ec59f092884a7b15b8018592>f0fb4...,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,explicit,155.0,4. Discussion,136.0,"Additionally, no sex differences were detected...",result,3cf9c7cd259a356839f42ecf143af3a8f6ef8b54,74cbd6d0eeb051b036f806d8a86c3a85859f9d7d,False,3cf9c7cd259a356839f42ecf143af3a8f6ef8b54>74cbd...,3cf9c7cd259a356839f42ecf143af3a8f6ef8b54>74cbd...,0,supportive,NaN
1857,explicit,188.0,3. Discussion,184.0,WBRT (40Gy in 20 fractions) along with concurr...,background,e609824e9ea6bee5aca817238d81d1cdd6b462ad,f7bfdcf8892a561b6030ed541924551fb78acf1f,False,e609824e9ea6bee5aca817238d81d1cdd6b462ad>f7bfd...,e609824e9ea6bee5aca817238d81d1cdd6b462ad>f7bfd...,1,NaN,NaN
1858,explicit,110.0,Methods,103.0,The data obtained from this crosssectional sur...,method,19317f7188bc6ecad985c46277969c0ac03dbcf8,0c86f0d577f04534edc14a509a68ae80ce6fbb74,False,19317f7188bc6ecad985c46277969c0ac03dbcf8>0c86f...,19317f7188bc6ecad985c46277969c0ac03dbcf8>0c86f...,0,NaN,NaN
1859,explicit,360.0,Discussion,321.0,"Additionally, encapsulated spheroids may be mu...",background,62ac94ab9227b84f1317edad1b6312e311981961,df5084196ea93af9250fae27c981ea3d7959599d,False,62ac94ab9227b84f1317edad1b6312e311981961>df508...,62ac94ab9227b84f1317edad1b6312e311981961>df508...,1,NaN,NaN


In [5]:
def simplify_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def convert_synonyms(sentence: str) -> str:
    words = sentence.split()
    pos_tags = list(map(lambda x: (x[0], simplify_pos(x[1])), nltk.pos_tag(words)))

    new_sentence = []

    for i in range(len(words)):
        
        synset = wordnet.synsets(words[i], pos_tags[i][1])
        if words[i] not in stopwords.words("english") and synset:
            if (len(synset[0].lemmas()) > 1):
                synonym = synset[0].lemmas()[1].name()
            else:
                synonym = synset[0].lemmas()[0].name()
            new_sentence.append(synonym.replace("_", " "))
        else:
            new_sentence.append(words[i])
        
    return ' '.join(new_sentence)


In [7]:
synonym_df = X_test.apply(lambda x: convert_synonyms(x))
synonym_df

0       Chapel, as good as X10 [2], UPC [3] , CoArray ...
1       In addition, the effect of the present study b...
2       several instrument that more specifically addr...
3       Organotypic hippocampal piece civilization int...
4       actuate PBMC are the basis of the standard PBM...
                              ...                        
1856    Additionally, no sexual practice difference we...
1857    WBRT (40Gy in 20 fractions) along with coincid...
1858    The information obtain from this crosssectiona...
1859    Additionally, encapsulate ellipsoid of revolut...
1860    The dosage of L-NMMA and Indocin instill were ...
Name: string, Length: 1861, dtype: object

In [15]:
df = pd.DataFrame({"string": synonym_df, "label": y_test})

In [16]:
df.to_json(
    "synonymized.jsonl",
    lines=True,
    orient="records",
)